In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [2]:
def edu(x):
    if x == 2:
        return 6
    elif x == 3:
        return 9
    elif x == 4:
        return 12
    elif x == 5:
        return 14
    elif x == 6:
        return 16
    elif x == 7:
        return 18
    elif x == 8:
        return 21
    else:
        return 0
    

In [5]:
df = pd.read_csv("household_2020a.csv")
df['education_year'] = df['education'].apply(edu)
df= df[['age','number','education_year','income']]
df1 = df.loc[df['number'].isin([1])& (df['age'] >=30) & (df['age']<=39)]
df2 = df1.copy()
edu_model = ols("income~education_year",data=df2).fit()
print(edu_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     10.88
Date:                Mon, 10 Apr 2023   Prob (F-statistic):            0.00106
Time:                        14:49:45   Log-Likelihood:                -3474.5
No. Observations:                 388   AIC:                             6953.
Df Residuals:                     386   BIC:                             6961.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        714.1438    680.901      1.

# RMSE 계산
- 회귀계수의 표준오차를 구하는 데 필요한 RMSE(o)을 구합니다

In [6]:
intercept = edu_model.params[0]
slope = edu_model.params[1]
df2['fitted'] = intercept + df2['education_year'] * slope
df2['sse'] = (df2['income']-df2['fitted'])**2
sse = df2[['sse']]
n = len(df2.index)
k = 2
sigma_hat = np.sqrt(np.sum(sse)/(n-k))  ## RMSE
np.round(sigma_hat,2)


sse    1879.21
dtype: float64

# 회귀계수의 표준 오차
- RMSE와 다른 정보를 이용하여 절편과 기울기의 표준오차인 se(a), se(b)를 구합니다

In [7]:
df2

,age,number,education_year,income,fitted,sse
19,35,1,14,1033,2828.283960,3.223044e+06
134,30,1,16,3032,3130.303985,9.663673e+03
157,30,1,16,1812,3130.303985,1.737925e+06
190,36,1,16,3959,3130.303985,6.867371e+05
191,31,1,16,1197,3130.303985,3.737664e+06
...,...,...,...,...,...,...
17525,37,1,12,3090,2526.263936,3.177984e+05
17544,30,1,14,1017,2828.283960,3.280750e+06
17599,37,1,16,4603,3130.303985,2.168834e+06
17696,37,1,14,1122,2828.283960,2.911405e+06


In [16]:
df2['x_bar'] = df2['education_year'].mean()
df2['x_residual2'] = (df2['education_year']-df2['x_bar'])**2
x_bar = df2['x_bar'].mean()
x_bar = round(x_bar,2)
x_residual2 = np.sum(df2['x_residual2'])
n = len(df2.index)
se_a = np.sqrt(1/n + x_bar**2/x_residual2)*sigma_hat
se_b = sigma_hat / np.sqrt(x_residual2)
np.round(se_a,2)


sse    681.05
dtype: float64

In [17]:
np.round(se_b,2)

sse    45.78
dtype: float64

# 회귀계수의 신뢰구간
- a의 추정계수에 대한 95% 신뢰구간은 a+-2se(a)이며, B의 추정계수에 대한 95% 신뢰구간은 B+-2se(b)입니다

In [19]:
# a에 대한 신뢰구간
lb_a = intercept - 2 * se_a # lower bound
hb_a = intercept + 2 * se_a # higher bound
np.round(lb_a,2)


sse   -647.95
dtype: float64

In [20]:
np.round(hb_a,2)

sse    2076.24
dtype: float64

In [23]:
# B에 대한 신뢰구간
lb_b = slope - 2 * se_b
hb_b = slope + 2 * se_b
np.round(lb_b,2)


sse    59.45
dtype: float64

In [24]:
np.round(hb_b,2)

sse    242.57
dtype: float64

# 절편과 기울기에 대한 유의성 검증
- 단순회귀분석 모형의 절편과 기울기에 대한 귀무가설 B1 = B0 = 0에 대해 t-검정을 실시합니다

In [27]:
# 절편에 대한 t검정
t_a = (intercept-0)/se_a
t_a.round(3)

sse    1.049
dtype: float64

In [28]:
# 기울기에 대한 t검정
t_b = (slope-0)/se_b
t_b.round(4)

sse    3.2986
dtype: float64

# p값
- 절편과 기울기에 대한 귀무가설에 대한 가설에 대해 t값에 해당하는 p값을 구한다


In [29]:
from scipy import stats
df = len(df2.index)-2
p_a = (1 - stats.t.cdf(t_a,df=df))*2   # 대립가설을 a 는 0이 아니다로 둘 경우 p값
p_b = (1-stats.t.cdf(t_b,df=df))*2   # 대립가설을 B 는 0이 아니다로 둘 경우 p값
p_a.round(4)

array([0.295])

In [30]:
p_b.round(4)

array([0.0011])

# 대학 졸업 30대 1인 가구의 소득 예측
## 예측 추정치

- 회귀모형을 통해 구한 절편과 기울기를 이용하여 대학교육 연수에 해당하는 16년 교육을 받은 가구주의 소득을 추정한다.

In [35]:
def edu(x):
    if x == 2:
        return 6
    elif x == 3:
        return 9
    elif x == 4:
        return 12
    elif x == 5:
        return 14
    elif x == 6:
        return 16
    elif x == 7:
        return 18
    elif x == 8:
        return 21
    else:
        return 0
    

In [36]:
df = pd.read_csv("household_2020a.csv")
df['education_year'] = df['education'].apply(edu)
df= df[['age','number','education_year','income']]
df1 = df.loc[df['number'].isin([1])& (df['age'] >=30) & (df['age']<=39)]
df2 = df1.copy()
edu_model = ols("income~education_year",data=df2).fit()
print(edu_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     10.88
Date:                Mon, 10 Apr 2023   Prob (F-statistic):            0.00106
Time:                        15:33:36   Log-Likelihood:                -3474.5
No. Observations:                 388   AIC:                             6953.
Df Residuals:                     386   BIC:                             6961.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        714.1438    680.901      1.

In [37]:
intercept = edu_model.params[0]
coef1 = edu_model.params[1]

In [39]:
y_16 = intercept + coef1 * 16
y_16.round(1)

3130.3

## 평균값과 개별값에 대한 표준오차
- 대학졸업자의 평균적인 소득의 표준오차는 

In [40]:
x_0 = 16  # 대학졸업에 해당하는 교육연수

df2 = df1.copy()
df2['fitted'] = intercept + df2['education_year']*coef1
df2['sse'] = (df2['income']-df2['fitted'])**2
df3 = df2['sse']
n = len(df3.index)
df4 = df3.sum(axis=0)
sigma_hat = np.sqrt(df4/(n-2)) # RMSE




In [43]:
df2['x_bar'] = df2['education_year'].mean()
x_bar = df2['education_year'].mean()
df2['x_residual2'] = (df2['education_year']-df2['x_bar'])**2
df3 = df2['x_residual2']
df4 = df3.sum(axis=0)
x_residual2= df4

# 30대 1인 가구의 대학졸업자의 소득의 평균예측치의 표준오차
se1_16 = np.sqrt(1/n+(x_0 - x_bar)**2/x_residual2) * sigma_hat

np.round(se1_16,2)

111.8

- 30대 1인 가구의 대학졸업자의 평균적인 소득의 95% 신뢰구간은 =>

In [44]:
lb = y_16 - 2 * se1_16  # lower bound
hb = y_16 + 2 * se1_16 # higher bound
lb.round(2)

2906.71

In [45]:
hb.round(2)

3353.9

- 30대 1인 가구 중 특정 대학 졸업자 가구의 소득의 표준오차를 다음과 같이 예측한다.

In [47]:
# 30대1인 가구인 대학졸업자의 소득의 평균예측치의 표준오차
se2_16 = np.sqrt(1+1/n + (x_0 -x_bar)**2/x_residual2) * sigma_hat


In [49]:
lb2 = y_16 - 2*se2_16
hb2 = y_16 + 2*se2_16
lb2.round(2)

-634.76

In [50]:
hb2.round(2)

6895.37